# Gamfort's Locally Weighted Regression

---



In [19]:
# Graph configuration
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
from IPython.display import Image
from IPython.display import display
plt.style.use('seaborn-white')

In [20]:
# computational libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats 
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [21]:
# Uniform functions

lm = LinearRegression()
scale = StandardScaler()
qscale = QuantileTransformer()

Load in data
---

In [30]:
data = pd.read_csv("/content/drive/MyDrive/Data Science/Data 441/LOWESS/diamonds.csv").iloc[:,1:].sample(1000)
data

,carat,cut,color,clarity,depth,table,price,x,y,z
34829,0.30,Premium,H,VVS1,62.2,58.0,878,4.29,4.26,2.66
876,0.77,Very Good,E,SI1,63.2,58.0,2873,5.80,5.84,3.68
41360,0.38,Ideal,H,VVS1,60.7,55.0,1219,4.71,4.74,2.87
8374,0.31,Premium,F,VS2,59.4,58.0,583,4.37,4.45,2.62
29402,0.34,Ideal,F,VS2,60.8,57.0,699,4.51,4.54,2.75
...,...,...,...,...,...,...,...,...,...,...
37504,0.41,Premium,H,SI1,61.8,59.0,984,4.81,4.77,2.96
27182,2.20,Ideal,H,VS2,61.7,55.0,17460,8.41,8.38,5.18
28556,0.30,Premium,H,VS1,60.4,61.0,675,4.34,4.30,2.61
19374,0.27,Premium,E,VVS1,60.6,58.0,622,4.18,4.20,2.54


For this experiment we will select multiple variables: our dependent vairale (y) will be price, and our independent variables will be carat, depth and table.

In [50]:
X, y = data[['carat', 'depth', 'table']].values, data['price'].values

In [51]:
# Scale our X values using StandardScalar()
X = scale.fit_transform(X)

Gramfort's LOWESS
---

*from class notebook

This implementation only works with single dimensional datasets.

In [33]:
# approach by Alex Gramfort --> only for 1-dimensional inputs

# https://gist.github.com/agramfort/850437

"""William S. Cleveland: "Robust locally weighted regression and smoothing
scatterplots", Journal of the American Statistical Association, December 1979,
volume 74, number 368, pp. 829-836.

William S. Cleveland and Susan J. Devlin: "Locally weighted regression: An
approach to regression analysis by local fitting", Journal of the American
Statistical Association, September 1988, volume 83, number 403, pp. 596-610."""

def lowess_ag(x, y, f=2. / 3., iter=3):
    """lowess(x, y, f=2./3., iter=3) -> yest
    Lowess smoother: Robust locally weighted regression.
    The lowess function fits a nonparametric regression curve to a scatterplot.
    The arrays x and y contain an equal number of elements; each pair
    (x[i], y[i]) defines a data point in the scatterplot. The function returns
    the estimated (smooth) values of y.
    The smoothing span is given by f. A larger value for f will result in a
    smoother curve. The number of robustifying iterations is given by iter. The
    function will run faster with a smaller number of iterations.
    """
    n = len(x)
    r = int(ceil(f * n))
    h = [np.sort(np.abs(x - x[i]))[r] for i in range(n)]
    w = np.clip(np.abs((x[:, None] - x[None, :]) / h), 0.0, 1.0)
    w = (1 - w ** 3) ** 3
    yest = np.zeros(n)
    delta = np.ones(n)
    for iteration in range(iter):
        for i in range(n):
            weights = delta * w[:, i]
            b = np.array([np.sum(weights * y), np.sum(weights * y * x)])
            A = np.array([[np.sum(weights), np.sum(weights * x)],
                          [np.sum(weights * x), np.sum(weights * x * x)]])
            beta = linalg.solve(A, b)
            yest[i] = beta[0] + beta[1] * x[i]

        residuals = y - yest
        s = np.median(np.abs(residuals))
        delta = np.clip(residuals / (6.0 * s), -1, 1)
        delta = (1 - delta ** 2) ** 2

    return yest

Multi-Dimensional Gramfort's LOWESS
---
*from class notebook

This modified function allows us to apply Gamfort's model to a multivariate dataset.

In [34]:
#We need a function to calculate the Euclidean distance between our observations

def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [35]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True):

  n = len(x)
  r = int(ceil(f * n))
  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)
  
  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]

  w = np.clip(dist(x,x) / h, 0.0, 1.0)
  w = (1 - w ** 3) ** 3

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(w[:,i])
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization
      beta = linalg.solve(A, b)
      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta)

    residuals = y - yest
    s = np.median(np.abs(residuals))
    delta = np.clip(residuals / (6.0 * s), -1, 1)
    delta = (1 - delta ** 2) ** 2

  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
  else:
    f = LinearNDInterpolator(x, yest)
  output = f(xnew) # the output may have NaN's where the data points from xnew are outside the convex hull of X
  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,y.ravel()) 
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [62]:
# Intialize our estimate matrix and training and test sets
X_train, X_test, y_train, y_test = tts(X,y,test_size=0.3,shuffle=True,random_state=123)

In [69]:
# Make our estimation using the multidimensional lowess
yhat = lw_ag_md(X_train,y_train,X_test,f=1/3,iter=7,intercept=True)

In [70]:
# Take the mean squared error
mse(y_test, yhat)

2157369.2877384936

we see here that our mse is not particularly great

K-Fold Cross Validation Testing

In [71]:
kf = KFold(n_splits=10,shuffle=True,random_state=123)

In [75]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)

for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  yhat_lw = lw_ag_md(xtrain,ytrain,xtest,f=1/3,iter=7,intercept=True)
  
  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 2367480.843709505
The Cross-validated Mean Squared Error for Random Forest is : 2352212.9550595093


Scikit Compliant Functions
---

In [76]:
class Lowess_AG_MD:
    def __init__(self, f = 1/10, iter = 3,intercept=True):
        self.f = f
        self.iter = iter
        self.intercept = intercept
    
    def fit(self, x, y):
        f = self.f
        iter = self.iter
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        f = self.f
        iter = self.iter
        intercept = self.intercept
        return lw_ag_md(x, y, x_new, f, iter, intercept)

    def get_params(self, deep=True):
    # suppose this estimator has parameters "f", "iter" and "intercept"
        return {"f": self.f, "iter": self.iter,"intercept":self.intercept}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [77]:
model = Lowess_AG_MD(f=1/5,iter=3,intercept=True)
model.fit(xtrain,ytrain)
yhat = model.predict(xtest)
mse(ytest,yhat)

1971553.6296132214

K-Fold validation

In [78]:
mse_lwr = []
mse_rf = []
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
model_rf = RandomForestRegressor(n_estimators=200,max_depth=5)
model_lw = Lowess_AG_MD(f=1/65,iter=3,intercept=True)

for idxtrain, idxtest in kf.split(X):
  xtrain = X[idxtrain]
  ytrain = y[idxtrain]
  ytest = y[idxtest]
  xtest = X[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_lw.fit(xtrain,ytrain)
  yhat_lw = model_lw.predict(xtest)
  
  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for Random Forest is : '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 2737352.0659849485
The Cross-validated Mean Squared Error for Random Forest is : 2258511.4118019794


Gridsearch

---

In [79]:
lwr_pipe = Pipeline([('zscores', StandardScaler()),
                     ('lwr', Lowess_AG_MD())])

params = [{'lwr__f': [1/i for i in range(3,15)],
         'lwr__iter': [1,2,3,4]}]

gs_lowess = GridSearchCV(lwr_pipe,
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)
gs_lowess.fit(X, y)
gs_lowess.best_params_

{'lwr__f': 0.3333333333333333, 'lwr__iter': 1}

In [81]:
gs_lowess.score(X,y)

-2024384.669648938